In [4]:
import pandas as pd

# Log path
log_path = 'log.txt'

# Read log data into df
log_df = pd.read_csv(log_path, sep='\t', header=None, names=[ 'beaconflag', 'timestamp', 'beaconId', 'beaconPos', 'angleGroundTruth', 'angleDistorted', 'sensorSTD',
    'headingAngleGroundTruth', 'headingAngleDistorted', 'compassSTD'])

# Keep only relevant data
relevant_columns = ['timestamp', 'beaconPos', 'angleGroundTruth', 'angleDistorted', 'headingAngleGroundTruth', 'headingAngleDistorted']
filtered_df = log_df[relevant_columns]

# Show filtered
print(filtered_df.head())


       timestamp             beaconPos  angleGroundTruth  angleDistorted  \
0  1705931099806   (30.00, 0.00, 0.00)         313.45070       313.80750   
1  1705931100009  (-30.00, 0.00, 0.00)          47.40670        46.12626   
2  1705931100227   (30.00, 0.00, 0.00)         311.60500       311.79470   
3  1705931100626  (-30.00, 0.00, 0.00)          50.42073        47.23244   
4  1705931100647   (30.00, 0.00, 0.00)         309.46560       310.56270   

   headingAngleGroundTruth  headingAngleDistorted  
0                        0              -1.604125  
1                        0               0.842163  
2                        0              -0.488708  
3                        0               0.051735  
4                        0              -0.182267  


In [2]:
# Define columns we want:
# Timestamp, AngleBeacon0, STDBeacon0, AngleBeacon1, STDBeacon1

wl = []

for index, row in log_df.iterrows():
    if row['beaconId'] == 0:
        wl.append([row['timestamp'], row['angleDistorted'], row['sensorSTD'], 0, 1000])
    elif row['beaconId'] == 1:
        wl.append([row['timestamp'], 0, 1000, row['angleDistorted'], row['sensorSTD']])

wl_df = pd.DataFrame(wl, columns=['timestamp', 'AngleBeacon0', 'StdBeacon0', 'AngleBeacon1', 'StdBeacon1'])

# As array
#print(wl)

# As dataframe
#print(wl_df.head())

# --- --- ---

# In case, convert into numpy array
wl_np = wl_df.to_numpy()

# Show content
print(wl_np)

[[1.70593110e+12 0.00000000e+00 1.00000000e+03 3.13807500e+02
  3.00000000e+00]
 [1.70593110e+12 4.61262600e+01 3.00000000e+00 0.00000000e+00
  1.00000000e+03]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 3.11794700e+02
  3.00000000e+00]
 [1.70593110e+12 4.72324400e+01 3.00000000e+00 0.00000000e+00
  1.00000000e+03]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 3.10562700e+02
  3.00000000e+00]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 3.08949900e+02
  3.00000000e+00]
 [1.70593110e+12 5.89843100e+01 3.00000000e+00 0.00000000e+00
  1.00000000e+03]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 3.04894500e+02
  3.00000000e+00]
 [1.70593110e+12 5.48897100e+01 3.00000000e+00 0.00000000e+00
  1.00000000e+03]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 2.98653000e+02
  3.00000000e+00]
 [1.70593110e+12 0.00000000e+00 1.00000000e+03 3.00413300e+02
  3.00000000e+00]
 [1.70593110e+12 5.80936800e+01 3.00000000e+00 0.00000000e+00
  1.00000000e+03]
 [1.70593110e+12 0.00000000e+00 1.000000

In [3]:
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints
import numpy as np
import pandas as pd

def f_x(state, dt):
    """
    Zustandsübergangsfunktion.
    Da die Geschwindigkeit konstant ist und der Zustand nur die Position beinhaltet,
    hat diese Funktion einfach die Geschwindigkeit zur Position hinzugefügt.
    TODO: Später Orientierungswinkel rein (und Winkel in der Messfunktion beachten)
    """  
    x, y, vx, vy = state

    # Update position with current velocity
    new_x = x + vx * dt
    new_y = y + vy * dt

    # Speed stays constant (no acceleration yet)
    new_vx = vx
    new_vy = vy

    return np.array([new_x, new_y, new_vx, new_vy])


def h_x(state):
    """
    Messfunktion.
    Bildet den Zustandsraum (Position und Geschwindigkeit des Schiffs)
    auf den Messraum ab (Winkel von den Funktürmen), wobei definiert als:
    - Norden als 0°, 
    - Osten als 270°,
    - Süden als 180°,
    - Westen als 90°.
    """
    x, y, _, _ = state
    
    # Positionen der Beacons
    beacon0_pos = np.array([-30, 0])
    beacon1_pos = np.array([30, 0])

    # Berechne den Winkel vom Schiff zu jedem Beacon (Konvertierung von Radians in Winkel)
    angle0 = np.arctan2(y - beacon0_pos[1], x - beacon0_pos[0]) * 180 / np.pi
    angle1 = np.arctan2(y - beacon1_pos[1], x - beacon1_pos[0]) * 180 / np.pi

    # Anpassung der Winkel an definierte Orientierung (Orientierungswinkel später ebenfalls implementieren)
    angle0 = (angle0 + 270) % 360
    angle1 = (angle1 + 270) % 360

    return np.array([angle0, angle1])

    
def test_hx():
    """
    Testfunktion für die Messfunktion.
    Überprüft die Winkelberechnung für verschiedene Positionen des Schiffs relativ zu den Funktürmen.
    """
    test_states = [
        np.array([0, 0, 0, 0]),  # Ship right between both beacons; 270  90
        np.array([0, 30, 0, 0]),  # Ship north from beacon0 and beacon1
        np.array([0, -30, 0, 0]), # Ship south from beacon0 and beacon1
        np.array([-60, 0, 0, 0]), # Schiff west from beacon0
        np.array([60, 0, 0, 0])   # Schiff east from beacon1
    ]

    for state in test_states:
        angles = h_x(state)
        print(f"Position from ship: {state[:2]}: Angle to beacon0: {angles[0]}° - Angle to beacon1: {angles[1]}°")


# Define the initial state (position of the ship)
initial_state = np.array([0, -30])  # Position at (0, -30)

# Define the speed of the ship (constant)
speed = 0.02  # in unity unit (float)

# Time step (dt) - assuming 1 time unit (ms) per iteration
dt = 1

# Define sigma points
# n =
# alpha =
# beta =
# kappa =
sigma_points = MerweScaledSigmaPoints(n=4, alpha=0.1, beta=2., kappa=1.)

# Initialize Unscented Kalman Filter
ukf = UnscentedKalmanFilter(dim_x=4, dim_z=2, dt=dt, fx=f_x, hx=h_x, points=sigma_points)

# Set the initial state
ukf.x = initial_state

# Measurement noise (standard deviation of the sensor accuracy)
measurement_std = 3  # in degrees (std)

# Set measurement noise covariance matrix
# TODO: Clarifying value
ukf.R = np.diag([measurement_std**2, measurement_std**2])

# Process noise covariance matrix
ukf.Q = np.eye(4) * 0.01

# Initial setup of the UKF
ukf

# Load log data
log_data = pd.read_csv('log.txt', sep='\t', names=['beaconflag', 'timestamp', 'beaconId', 'beaconPos', 'angleGroundTruth', 'angleDistorted', 'sensorSTD',
    'headingAngleGroundTruth', 'headingAngleDistorted', 'compassSTD'])


log_data.head()

# Testfunktion
test_hx()

Position from ship: [0 0]: Angle to beacon0: 270.0° - Angle to beacon1: 90.0°
Position from ship: [ 0 30]: Angle to beacon0: 315.0° - Angle to beacon1: 45.0°
Position from ship: [  0 -30]: Angle to beacon0: 225.0° - Angle to beacon1: 135.0°
Position from ship: [-60   0]: Angle to beacon0: 90.0° - Angle to beacon1: 90.0°
Position from ship: [60  0]: Angle to beacon0: 270.0° - Angle to beacon1: 270.0°
